In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/sc_lab_assignment.zip', 'r') as zipObj:
     zipObj.extractall('sc_lab_assignment')

In [3]:
pip install split-folders[full]

In [4]:
import splitfolders

In [7]:
input_folder = '/content/sc_lab_assignment'

In [9]:
splitfolders.ratio(input_folder, output="/content/drive/MyDrive/splited_dataset(sc_lab_assignment)",
                   seed=42, ratio=(.7, .2,.1),
                   group_prefix=None)


Copying files: 0 files [00:00, ? files/s]
Copying files: 9 files [00:00, 88.10 files/s]
Copying files: 23 files [00:00, 116.83 files/s]
Copying files: 37 files [00:00, 125.95 files/s]
Copying files: 51 files [00:00, 128.76 files/s]
Copying files: 65 files [00:00, 132.45 files/s]
Copying files: 79 files [00:00, 131.82 files/s]
Copying files: 94 files [00:00, 137.05 files/s]
Copying files: 108 files [00:00, 137.66 files/s]
Copying files: 122 files [00:00, 123.18 files/s]
Copying files: 135 files [00:01, 103.26 files/s]
Copying files: 146 files [00:01, 79.55 files/s] 
Copying files: 156 files [00:01, 82.25 files/s]
Copying files: 166 files [00:01, 84.81 files/s]
Copying files: 179 files [00:01, 94.02 files/s]
Copying files: 193 files [00:01, 98.72 files/s]
Copying files: 205 files [00:01, 103.35 files/s]
Copying files: 216 files [00:02, 93.28 files/s] 
Copying files: 227 files [00:02, 97.44 files/s]
Copying files: 238 files [00:02, 99.43 files/s]
Copying files: 249 files [00:02, 99.07 fi

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [11]:
import os

In [12]:
train_path="/content/drive/MyDrive/splited_dataset(sc_lab_assignment)/train"
test_path="/content/drive/MyDrive/splited_dataset(sc_lab_assignment)/test"
valid_path="/content/drive/MyDrive/splited_dataset(sc_lab_assignment)/val"

In [13]:
# Define data transformations (you can add more augmentations if required)
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the images to a fixed size
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
])

In [14]:
train_data = torchvision.datasets.ImageFolder(train_path, transform=data_transforms)
val_data = torchvision.datasets.ImageFolder(valid_path, transform=data_transforms)
test_data = torchvision.datasets.ImageFolder(test_path, transform=data_transforms)

In [15]:
print(f"Number of samples in training set: {len(train_data)}")
print(f"Number of samples in testing set: {len(test_data)}")
print(f"Number of samples in validation set: {len(val_data)}")

Number of samples in training set: 3658
Number of samples in testing set: 524
Number of samples in validation set: 1045


In [35]:
sequence_length = 256
input_size =256*3
hidden_size = 128
num_layers = 2
num_classes= 3
batch_size = 62
num_iters = 1475
learning_rate = 0.01  # More power so we can learn faster! previously it was 0.001

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [36]:
num_epochs = num_iters / (len(train_data) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

25


In [37]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True,drop_last=True)   # It's better to shuffle the whole training dataset!

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=batch_size,
                                          shuffle=False,drop_last=True)
val_loader = torch.utils.data.DataLoader(dataset=val_data,
                                          batch_size=batch_size,
                                          shuffle=False,drop_last=True)

In [38]:
image_idx = 0  # Index of the image you want to check
image, _ = train_data[image_idx]  # Load the image at the specified index

# Get the dimensions (size) of the image
image_size = image.shape
print("Image size:", image_size)

Image size: torch.Size([3, 256, 256])


In [39]:
torch.manual_seed(30)

In [40]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size= hidden_size
        self.num_layers = num_layers

        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # For uni Directional LSTM
        #self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) # For uni Directional RNN
        #self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True,bidirectional=True) # For BiDirectional RNN
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,bidirectional=True) # For BiDirectional LSTM
        #self.fc = nn.Linear(hidden_size, num_classes) #For uni Directional
        self.fc = nn.Linear(hidden_size*2, num_classes) #For Bidirectional

    def forward(self, x):
         #set initial hidden and cell states
        #h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) #For uni Directional
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) #For uni Directional
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) #For Bidirectional
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) #For Bidirectional

        #Forward Propagation
        #out, _  = self.rnn(x,h0)
        out, _  = self.lstm(x,(h0,c0)) #out: tensor of shape (batch size, seq_length, hidden_size)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [41]:
'''
INSTANTIATE MODEL CLASS
'''
model = RNN( input_size, hidden_size, num_layers, num_classes)
# To enable GPU
model.to(device)

RNN(
  (lstm): LSTM(768, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=3, bias=True)
)

In [42]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [43]:
'''
TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 59 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in val_loader:

                images = images.reshape(-1, sequence_length, input_size).to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Val Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 59. Loss: 0.8101324439048767. Val Accuracy: 63.306451612903224
Iteration: 118. Loss: 0.9174693822860718. Val Accuracy: 63.00403225806452
Iteration: 177. Loss: 0.7148790955543518. Val Accuracy: 71.06854838709677
Iteration: 236. Loss: 0.5799575448036194. Val Accuracy: 74.39516129032258
Iteration: 295. Loss: 0.4771195650100708. Val Accuracy: 75.40322580645162
Iteration: 354. Loss: 0.4752984642982483. Val Accuracy: 74.09274193548387
Iteration: 413. Loss: 0.514603853225708. Val Accuracy: 78.2258064516129
Iteration: 472. Loss: 0.5935910940170288. Val Accuracy: 78.52822580645162
Iteration: 531. Loss: 0.30609893798828125. Val Accuracy: 78.93145161290323
Iteration: 590. Loss: 0.5109290480613708. Val Accuracy: 79.13306451612904
Iteration: 649. Loss: 0.433707058429718. Val Accuracy: 79.03225806451613
Iteration: 708. Loss: 0.46328413486480713. Val Accuracy: 77.21774193548387
Iteration: 767. Loss: 0.5181794762611389. Val Accuracy: 80.44354838709677
Iteration: 826. Loss: 0.406414598226547

In [44]:
            correct = 0
            total = 0
            model.eval()
            # Iterate through test dataset
            for images, labels in test_loader:

                images = images.reshape(-1, sequence_length, input_size).to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print(f"Accuracy on the test set: {accuracy:.2f}%")

Accuracy on the test set: 83.27%
